## 3.4 TensorFlow 实现神经网络

In [1]:
import tensorflow as tf
from numpy.random import RandomState

### 3.4.1 前向传播算法

简单线性模型

$$
W^{(1)} = \begin{bmatrix}
W_{1,1}^{(1)} & W_{1,2}^{(1)} & W_{1,3}^{(1)} \\
W_{2,1}^{(1)} & W_{2,2}^{(1)} & W_{2,3}^{(1)} \\
\end{bmatrix}
$$

$$
a^{(1)} = \begin{bmatrix}
a_{1,1} & a_{1,2} & a_{1,3} \\
\end{bmatrix} = xW^{(1)} = \begin{bmatrix}
x_1 & x_2 \\
\end{bmatrix} \begin{bmatrix}
W_{1,1}^{(1)} & W_{1,2}^{(1)} & W_{1,3}^{(1)} \\
W_{2,1}^{(1)} & W_{2,2}^{(1)} & W_{2,3}^{(1)} \\
\end{bmatrix}
$$

$$
W^{(2)} = \begin{bmatrix}
W_{1,1}^{(2)} \\
W_{2,1}^{(2)} \\
W_{3,1}^{(2)} \\
\end{bmatrix}
$$

$$
\begin{bmatrix}
y \\
\end{bmatrix} = a^{(1)}W^{(2)} = \begin{bmatrix}
a_{1,1} & a_{1,2} & a_{1,3} \\
\end{bmatrix} \begin{bmatrix}
W_{1,1}^{(2)} \\
W_{2,1}^{(2)} \\
W_{3,1}^{(2)} \\
\end{bmatrix}
$$

In [2]:
batch_size = 8

w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

x = tf.placeholder(tf.float32, shape=(None, 2), name="x-input")
y_ = tf.placeholder(tf.float32, shape=(None, 1), name="y-input")

a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

In [3]:
# 损失函数
cross_entry = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))

# 训练函数
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entry)

In [4]:
rdm = RandomState(1)
dataset_size = 128

X = rdm.rand(dataset_size, 2)
Y = [[int(x1 + x2 < 1)] for x1, x2 in X]

In [5]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    print(sess.run(w1))
    print(sess.run(w2))
    
    STEPS = 10000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        
        sess.run(train_step, feed_dict={
            x: X[start:end],
            y_: Y[start:end],
        })
        
        if i % 1000 == 0:
            total_cross_entryopy = sess.run(cross_entry, feed_dict={
                x: X,
                y_: Y,
            })
            print("%d: %g" % (i, total_cross_entryopy))
        
    print(sess.run(w1))
    print(sess.run(w2))

[[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]]
[[-0.81131822]
 [ 1.48459876]
 [ 0.06532937]]
0: 0.0674925
1000: 0.0163385
2000: 0.00907547
3000: 0.00714436
4000: 0.00578471
5000: 0.00430222
6000: 0.00280812
7000: 0.00137464
8000: 2.11566e-05
9000: -0
[[-2.59392238  3.18602753  2.38825655]
 [-4.1101799   1.68263638  2.83427358]]
[[-2.4300375 ]
 [ 3.33411145]
 [ 2.10067439]]


In [6]:
with tf.Session() as sess:
    v = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    print(tf.clip_by_value(v, 2.5, 4.5).eval(session=sess))

[[ 2.5  2.5  3. ]
 [ 4.   4.5  4.5]]


In [7]:
with tf.Session() as sess:
    v = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    print(tf.log(v).eval(session=sess))

[[ 0.          0.69314718  1.09861231]
 [ 1.38629436  1.60943794  1.79175949]]


In [8]:
with tf.Session() as sess:
    v0 = tf.constant([5.0, 1.0, 3.0])
    v1 = tf.constant([4.0, 6.0, 2.0])
    print(sess.run(v0 - v1))
    
    print(sess.run(tf.square(v0 - v1)))
    
    mse = tf.reduce_mean(tf.square(v0 - v1))
    print(sess.run(mse))
    
    loss = tf.reduce_sum(tf.where(tf.greater(v0, v1), v0 - v1, v1 - v0))
    print(sess.run(loss))

[ 1. -5.  1.]
[  1.  25.   1.]
9.0
7.0


### 4.4.2 过拟合
正则化：添加描述模型复杂度的函数

#### L1 正则化
$$ R(w) = \left|\left| w \right|\right|_1 = \sum_i \left| w_i \right| $$

#### L2  正则化
$$ R(w) = \left|\left| w \right|\right|_2^2 = \sum_i \left| w_i^2 \right| $$

In [9]:
with tf.Session() as sess:
    weights = tf.constant([[1.0, -2.0], [-3.0, 4.0]])
    
    loss = tf.contrib.layers.l1_regularizer(0.5)(weights)
    print(sess.run(loss))
    
    loss = tf.contrib.layers.l2_regularizer(0.5)(weights)
    print(sess.run(loss))

5.0
7.5
